In [212]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [213]:
trainpath = "/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Trainpath"
predpath = "/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath"

In [214]:
rootdir = pathlib.Path(trainpath)
classes = sorted([j.name.split("/")[-1] for j in rootdir.iterdir()])
print(classes)

['no', 'yes']


In [215]:
class Network(nn.Module):
    def __init__(self, num_classes = 2):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(num_features = 12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size = 2)
        
        self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 20, kernel_size = 3, stride = 1, padding = 1)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels = 20, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.bn3 = nn.BatchNorm2d(num_features = 32)
        self.relu3 = nn.ReLU()
        
        self.fc = nn.Linear(in_features = 32*50*50, out_features = num_classes)
        
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        
        output = self.pool(output)
        
        output = self.conv2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        
        output = output.view(-1, 32*50*50)
        output = self.fc(output)
        
        return output

In [216]:
checkpoint = torch.load("best_checkpoint.model")
model = Network(num_classes = 2)
model.load_state_dict(checkpoint)
model.eval()

Network(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=80000, out_features=2, bias=True)
)

In [241]:
transformer = transforms.Compose([
    transforms.Resize(100),
    # transforms.Lambda(lambda x: x.repeat(3,50,50)),
    transforms.CenterCrop(100),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [218]:
def prediction(img_path, transformer):
    image = Image.open(img_path)
    image_tensor = transformer(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    output = model(input)
    index = output.data.numpy().argmax()
    pred = classes[index]
    return pred

In [219]:
images_path = glob.glob(predpath + "/*.jpg")
images_path

['/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/10 no.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/15 no.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/17 no.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/18 no.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/3 no.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/8 no.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/9 no.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/Y1.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/Y10.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/Y11.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/Y12.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/Y17.jpg',
 '/media/mayi/Storage HDD/Summer Project/Brain tumor s

In [242]:
pred_dict = {}
for i in images_path:
    pred_dict[i[i.rfind('/')+1:]] = prediction(i, transformer)

RuntimeError: output with shape [1, 100, 100] doesn't match the broadcast shape [3, 100, 100]